## Implementation of AlexNet

### Loading data

### From a TensorFlow dataset

In [325]:
!pip install -q tensorflow tensorflow-datasets matplotlib

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

ImportError: Traceback (most recent call last):
  File "/home/ivan/tools/python_venv/python36/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/ivan/tools/python_venv/python36/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/ivan/tools/python_venv/python36/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/ivan/tools/python36/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/ivan/tools/python36/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.10.0: cannot open shared object file: No such file or

In [327]:
tfds.list_builders()

['bair_robot_pushing_small',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cifar10',
 'cifar100',
 'coco2014',
 'diabetic_retinopathy_detection',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'fashion_mnist',
 'image_label_folder',
 'imagenet2012',
 'imdb_reviews',
 'lm1b',
 'lsun',
 'mnist',
 'moving_mnist',
 'nsynth',
 'omniglot',
 'open_images_v4',
 'quickdraw_bitmap',
 'squad',
 'starcraft_video',
 'svhn_cropped',
 'tf_flowers',
 'wmt_translate_ende',
 'wmt_translate_enfr']

In [328]:
def getDatasetsImageNet():
    imageNetBuilder = tfds.builder("imagenet2012")
    imageNetBuilder.download_and_prepare()
    return ({
        'train': imageNetBuilder.as_dataset(split=tfds.Split.TRAIN), 
        'valid': imageNetBuilder.as_dataset(split=tfds.Split.VALIDATION),
        'test': imageNetBuilder.as_dataset(split=tfds.Split.TEST)})

In [329]:
#imageNet = getDatasetsImageNet()

### My dataset

In [1]:
import os
import shutil
import tensorflow as tf

In [2]:
def collectRawImgList(inputDir, outputDir):
    result = {'pathnames': list(), 'labelStrings': list(), 'newPathnames': list()}
    cnt = 0
    with os.scandir(inputDir) as inp:
        for entry in inp:
            if not os.path.isdir(entry):
                continue
            print("Processing directory {}".format(os.path.abspath(entry)))
            category = os.path.basename(entry)
            print("Category name: {}".format(category))
            for fileEntry in os.scandir(entry):
                #print("File {}".format(os.path.basename(fileEntry)))
                #result.append((os.path.abspath(fileEntry), category))
                result['pathnames'].append(os.path.abspath(fileEntry))
                result['labelStrings'].append(category)
                result['newPathnames'].append(os.path.join(outputDir, "{:05}.png".format(cnt)))
                cnt += 1
                
    return result

In [6]:
def formFileNameDataSet(inputDir, outputDir):
    rawCollection = collectRawImgList(inputDir, outputDir)
    
    placePathnames = tf.placeholder(shape=(None,), dtype=tf.string)
    placeLabelStrings = tf.placeholder(shape=(None,), dtype=tf.string)
    placeNewPathnames = tf.placeholder(shape=(None,), dtype=tf.string)
    
    dataSetPathnames = tf.data.Dataset.from_tensor_slices(placePathnames)
    dataSetLabelStrings = tf.data.Dataset.from_tensor_slices(placeLabelStrings)
    dataSetNewPathnames = tf.data.Dataset.from_tensor_slices(placeNewPathnames)
    
    dataset = tf.data.Dataset.zip((dataSetPathnames, dataSetLabelStrings, dataSetNewPathnames))
    
    return (dataset, rawCollection, (placePathnames, placeLabelStrings, placeNewPathnames))

In [7]:
def transformFiles(inputDir, outputDir, targetHeight=127, targetWidth=127):
    
    if os.path.exists(outputDir):
        shutil.rmtree(outputDir)
        
    os.mkdir(outputDir)
    outputDirOld = outputDir
    outputDir = os.path.join(outputDir, 'img')
    os.mkdir(outputDir)
    
    fnameDataset, rawCollection, placeholders = formFileNameDataSet(inputDir, outputDir)
    
    with open(os.path.join(outputDirOld, "fileLabels.txt"), "w") as labFile:
        for fn, lab in zip(rawCollection['newPathnames'], rawCollection['labelStrings']):
            labFile.write("'{}' {}\n".format(os.path.basename(fn), lab))
        
    
    def _howToTransform(pathname, labelString, newPathname):
        img = tf.image.resize_images(tf.image.decode_jpeg(tf.read_file(pathname)), 
                                     [targetHeight, targetWidth])
        #writeOp = tf.write_file(newPathname, tf.image.encode_png(tf.cast(img, tf.uint8)))
        return (pathname, labelString, newPathname, img)
    
    fnameDataset = fnameDataset.map(_howToTransform)
    iterator = fnameDataset.make_initializable_iterator()
    
    with tf.Session() as sess:
        sess.run(iterator.initializer, 
                 feed_dict={placeholders[0]: rawCollection['pathnames'],
                           placeholders[1]: rawCollection['labelStrings'],
                           placeholders[2]: rawCollection['newPathnames']})
        nextElement = iterator.get_next()
        writeOp = tf.write_file(nextElement[2], tf.image.encode_png(tf.cast(nextElement[3], tf.uint8)))
        while True:
            try:
                val, wop = sess.run([nextElement, writeOp])
                #print(val)
            except tf.errors.OutOfRangeError:
                break
                
    

In [8]:
transformFiles("/home/ivan/datasets/bilbao2019/raw",
               "/home/ivan/datasets/bilbao2019/processed", 
               targetHeight=227, targetWidth=227)

Processing directory /home/ivan/datasets/bilbao2019/raw/bench
Category name: bench
Processing directory /home/ivan/datasets/bilbao2019/raw/textad
Category name: textad
Processing directory /home/ivan/datasets/bilbao2019/raw/washbasin
Category name: washbasin
Processing directory /home/ivan/datasets/bilbao2019/raw/manhole
Category name: manhole
Processing directory /home/ivan/datasets/bilbao2019/raw/staircase
Category name: staircase
Processing directory /home/ivan/datasets/bilbao2019/raw/urn
Category name: urn
Processing directory /home/ivan/datasets/bilbao2019/raw/socket
Category name: socket
Processing directory /home/ivan/datasets/bilbao2019/raw/ad
Category name: ad
Processing directory /home/ivan/datasets/bilbao2019/raw/urinal
Category name: urinal
Processing directory /home/ivan/datasets/bilbao2019/raw/gate
Category name: gate
Processing directory /home/ivan/datasets/bilbao2019/raw/container
Category name: container
Processing directory /home/ivan/datasets/bilbao2019/raw/switch
Ca

Now we have prepared the dataset.

In [11]:
transformFiles("/home/ivan/datasets/bilbao2019/validation_raw",
               "/home/ivan/datasets/bilbao2019/validation_processed", 
               targetHeight=227, targetWidth=227)

Processing directory /home/ivan/datasets/bilbao2019/validation_raw/bench
Category name: bench
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/textad
Category name: textad
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/washbasin
Category name: washbasin
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/manhole
Category name: manhole
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/staircase
Category name: staircase
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/urn
Category name: urn
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/socket
Category name: socket
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/ad
Category name: ad
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/urinal
Category name: urinal
Processing directory /home/ivan/datasets/bilbao2019/validation_raw/gate
Category name: gate
Processing directory /home/ivan/datasets/bilba